In [90]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

attributes = [
    'Food Quality',
    'Portion Size',
    'Service (crew friendliness)',
    'Waiting time',
    'Aesthetics/Ambience'
]

wildcard_attributes = []
wildcard_attributes_all = [
    'Good Location (SG based)',
    'Good Food Presentation',
    'Ease of Payment',
    'Good Food Variety',
    'Accommodates Dietary Requirement',
    'Good Capacity',
    'Good Freebies',
    'Good Hygiene',
    'Good Marketing/Branding'
]

wildcard_selection = widgets.SelectMultiple(
    options=wildcard_attributes_all,
    # value=[],
    # rows=10,
    description='Wildcard Attributes:',
    disabled=False
)

TOTAL_CASH = 30
CASH = TOTAL_CASH

def create_slider(attr):
    int_slider = widgets.IntSlider(
        value=0,
        min=0,
        max=TOTAL_CASH,
        step=1,
        description=attr,
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        style= {'description_width': 'initial'}
    )
    int_slider.observe(on_value_change, names='value')
    int_slider.layout.width = '100%'
    return int_slider

def create_sliders():
    global sliders
    sliders = {}
    attributes_combined = attributes + list(wildcard_selection.value)

    # For each mandatory and wildcard attribute
    for attr in attributes_combined:
        # Create integer slider
        int_slider = create_slider(attr)
        sliders[attr] = int_slider
        
def reselect_sliders():
    global sliders, CASH
    
    attributes_to_remove = list(filter(
        lambda x: x not in attributes + list(wildcard_selection.value),
        sliders.keys()
    ))
    
    for attr in attributes_to_remove:
        CASH += sliders[attr].value
        sliders.pop(attr)
    
    attributes_to_add = list(filter(
        lambda x: x not in sliders.keys(),
        wildcard_selection.value
    ))
    
    for attr in attributes_to_add:
        int_slider = create_slider(attr)
        sliders[attr] = int_slider
    
    with sliders_box:
        reset_sliders_max()
        sliders_box.clear_output()
        display(widgets.VBox(list(sliders.values())))
    
    with chart_output:
        chart_output.clear_output()
        build_dataframe()
        plot_dataframe()

def build_dataframe():
    global df
    df = pd.DataFrame(
        index=['Affordability'] + list(sliders.keys()),
        data=[CASH] + [round(slider.value, 0) for slider in sliders.values()],
        columns=['Value']
    )
    
def plot_dataframe():
    x_pos=np.arange(df.shape[0])
    
    plt.bar(
        x=x_pos,
        height=df['Value'],
        color=['green'] + ['cornflowerblue' for _ in attributes] + ['yellow' for _ in range(len(sliders) - len(attributes))]
    )
    plt.ylim([0, TOTAL_CASH])
    plt.xticks(x_pos, df.index)
    plt.gcf().autofmt_xdate()
    plt.show()

def on_value_change(change):
    global CASH, df, sliders
    CASH = TOTAL_CASH - sum([slider.value for slider in sliders.values()])
    
    with remaining_cash_out:
        remaining_cash_out.clear_output()
        display(CASH)
        
    reset_sliders_max()
    
    with remaining_cash_out:
        display([slider.max for slider in sliders.values()])
        display([slider.layout.width for slider in sliders.values()])
    
    with chart_output:
        chart_output.clear_output()
        build_dataframe()
        plot_dataframe()
        
def on_wildcard_selection_change(change):
    global sliders
    reselect_sliders()
    
    with sliders_box:
        sliders_box.clear_output()
        display(widgets.VBox(list(sliders.values())))

def reset_sliders_max():
    global sliders, TOTAL_CASH, CASH
    new_sliders = []
    
    for slider in sliders.values():
        if CASH >= slider.max:
            slider.max = min(
                TOTAL_CASH,
                slider.value + CASH
            )
        elif CASH >= 0:
            slider.max = slider.value + CASH
        else:
            slider.max = slider.value

        slider.layout.width = f'{int(slider.max/ TOTAL_CASH* 100)}%'

create_sliders()
wildcard_selection.observe(on_wildcard_selection_change, names='value')

sliders_box = widgets.Output()
with sliders_box:
    display(widgets.VBox(list(sliders.values())))

remaining_cash_out = widgets.Output()
chart_output = widgets.Output()
with chart_output:
    build_dataframe()
    plot_dataframe()

wildcard_output = widgets.Output()
with wildcard_output:
    display(widgets.VBox([
        wildcard_selection
    ]))

widgets.VBox([
    chart_output,
    sliders_box,
    wildcard_output
])